<a href="https://colab.research.google.com/github/medsellufbc/BMI/blob/main/SEIR_P_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Define parameters with initial values
params = {
    'b': 0.00018,
    'mu': 4.563e-5,
    'mu_P': 0.1724,
    'beta_1': 0.00414,
    'beta_2': 0.0115,
    'delta': 0.7,
    'psi': 0.0051,
    'omega': 0.09,
    'sigma': 0.0018,
    'gamma_S': 0.05,
    'gamma_A': 0.0714,
    'eta_S': 0.1,
    'eta_A': 0.05,
    'alpha_1': 0.1,
    'alpha_2': 0.1,
    'sigma_9': 0.5,
    'sigma_10': 0.5,
    'sigma_11': 0.5,
    'sigma_12': 0.5,
    'sigma_13': 0.5,
    'sigma_14': 0.5
}

# Initial conditions
initial_conditions = {
    'S0': 93000,
    'E0': 1000,
    'IA0': 50,
    'IS0': 50,
    'R0': 0,
    'P0': 500
}

# Time steps
dt = 0.01
T = 200  # Total time
N = int(T / dt)
time = np.linspace(0, T, N)

# SEIRP model differential equations with stochastic terms
def f(y, t, params):
    S, E, IA, IS, R, P = y
    b = params['b']
    mu = params['mu']
    mu_P = params['mu_P']
    beta_1 = params['beta_1']
    beta_2 = params['beta_2']
    delta = params['delta']
    psi = params['psi']
    omega = params['omega']
    sigma = params['sigma']
    gamma_S = params['gamma_S']
    gamma_A = params['gamma_A']
    eta_S = params['eta_S']
    eta_A = params['eta_A']
    alpha_1 = params['alpha_1']
    alpha_2 = params['alpha_2']
    sigma_9 = params['sigma_9']
    sigma_10 = params['sigma_10']
    sigma_11 = params['sigma_11']
    sigma_12 = params['sigma_12']
    sigma_13 = params['sigma_13']
    sigma_14 = params['sigma_14']

    dSdt = b - (beta_1 * S * P) / (1 + alpha_1 * P) - (beta_2 * S * (IA + IS)) / (1 + alpha_2 * (IA + IS)) + psi * E - mu * S
    dEdt = (beta_1 * S * P) / (1 + alpha_1 * P) + (beta_2 * S * (IA + IS)) / (1 + alpha_2 * (IA + IS)) - psi * E - mu * E - omega * E
    dIAdt = (1 - delta) * omega * E - (mu + sigma * sigma_9) * IA - gamma_A * IA
    dISdt = delta * omega * E - (mu + sigma * sigma_10) * IS - gamma_S * IS
    dRdt = gamma_S * IS + gamma_A * IA - mu * R
    dPdt = eta_A * IA - mu_P * P + eta_S * IS - mu * P + sigma * sigma_11 * IA + sigma * sigma_12 * IS + sigma * sigma_13 * S + sigma * sigma_14 * E

    return np.array([dSdt, dEdt, dIAdt, dISdt, dRdt, dPdt])

# Runge-Kutta 4th order method with stochastic terms
def runge_kutta_stochastic(y, t, dt, params):
    s = 4  # Number of stages in RK4
    k = np.zeros((s, len(y)))  # Stages for deterministic part
    l = np.zeros((s, len(y)))  # Stages for stochastic part

    # Deterministic part stages
    k[0] = f(y, t, params)
    k[1] = f(y + 0.5 * dt * k[0], t + 0.5 * dt, params)
    k[2] = f(y + 0.5 * dt * k[1], t + 0.5 * dt, params)
    k[3] = f(y + dt * k[2], t + dt, params)

    # Stochastic part stages
    I_k = np.random.normal(size=(s, len(y)))  # Independent normal random variables
    l[0] = I_k[0] * np.array([params['sigma_9'], params['sigma_10'], params['sigma_11'], params['sigma_12'], params['sigma_13'], params['sigma_14']])
    l[1] = I_k[1] * np.array([params['sigma_9'], params['sigma_10'], params['sigma_11'], params['sigma_12'], params['sigma_13'], params['sigma_14']])
    l[2] = I_k[2] * np.array([params['sigma_9'], params['sigma_10'], params['sigma_11'], params['sigma_12'], params['sigma_13'], params['sigma_14']])
    l[3] = I_k[3] * np.array([params['sigma_9'], params['sigma_10'], params['sigma_11'], params['sigma_12'], params['sigma_13'], params['sigma_14']])

    # Combine deterministic and stochastic parts
    y_next = y + dt * (k[0] + 2*k[1] + 2*k[2] + k[3]) / 6
    y_next += np.sqrt(dt) * (l[0] + 2*l[1] + 2*l[2] + l[3]) / 6

    return y_next

# Function to run simulation and return results
def simulate_SEIRP(initial_conditions, params):
    S = np.zeros(N)
    E = np.zeros(N)
    IA = np.zeros(N)
    IS = np.zeros(N)
    R = np.zeros(N)
    P = np.zeros(N)

    S[0], E[0], IA[0], IS[0], R[0], P[0] = initial_conditions['S0'], initial_conditions['E0'], initial_conditions['IA0'], initial_conditions['IS0'], initial_conditions['R0'], initial_conditions['P0']

    for i in range(1, N):
        y = np.array([S[i-1], E[i-1], IA[i-1], IS[i-1], R[i-1], P[i-1]])
        y_next = runge_kutta_stochastic(y, time[i-1], dt, params)
        S[i], E[i], IA[i], IS[i], R[i], P[i] = y_next

    return S, E, IA, IS, R, P

# Interactive widgets for parameter adjustment
param_sliders = {}
for param_name, param_value in params.items():
    param_sliders[param_name] = widgets.FloatSlider(value=param_value, min=0, max=1, step=0.01, description=param_name)

initial_sliders = {}
for ic_name, ic_value in initial_conditions.items():
    initial_sliders[ic_name] = widgets.IntSlider(value=ic_value, min=0, max=100000, step=100, description=ic_name)

# Function to update parameters and plot
def update_plot(**kwargs):
    for key, value in kwargs.items():
        if key in params:
            params[key] = value
        elif key in initial_conditions:
            initial_conditions[key] = value

    S, E, IA, IS, R, P = simulate_SEIRP(initial_conditions, params)

    plt.figure(figsize=(14, 12))

    plt.subplot(321)
    plt.plot(time, S, label='Susceptible (S)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.subplot(322)
    plt.plot(time, E, label='Exposed (E)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.subplot(323)
    plt.plot(time, IA, label='Asymptomatic Infected (IA)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.subplot(324)
    plt.plot(time, IS, label='Symptomatic Infected (IS)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.subplot(325)
    plt.plot(time, R, label='Recovered (R)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.subplot(326)
    plt.plot(time, P, label='Protected (P)')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.legend()

    plt.tight_layout()
    plt.suptitle('SEIRP Model with Stochastic Terms', y=1.05)
    plt.show()

# Create interactive widgets
param_widgets = []
for param_name, slider in param_sliders.items():
    param_widgets.append(slider)

initial_widgets = []
for ic_name, slider in initial_sliders.items():
    initial_widgets.append(slider)

interactive_plot = widgets.interactive_output(update_plot, {
    **{param_name: slider for param_name, slider in param_sliders.items()},
    **{ic_name: slider for ic_name, slider in initial_sliders.items()}
})

display(widgets.VBox(param_widgets + initial_widgets))
display(interactive_plot)


Output()